# The Hockey Problem

In this notebook, I will explore data from the NHL (National Hockey League), which is a professional ice hockey league in North America (comprising teams from US and Canada). 

For this exercise, we have two sources of data:
1. https://www.scrapethissite.com/pages/forms
2. http://www.espn.com/nhl/statistics

I structured this notebook in a way it follows my thinking process :) 

# Libraries

In [80]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Reading data

### Web scrapping

In order to get the database of NHL team stats, let's use a web scraper using Beautiful Soup. For this, I modified the code from https://github.com/turing-usp/Trainee-DS-2021/blob/main/Scraping.ipynb.

In [81]:
def clean_names(names):
    """Clean line breaks \n 

    Keyword arguments:
    names -- list of names to be cleaned
    """
    
    name_list = []
    for i in range(len(names)):
        name = names[i].text.split()[0]
        for j in np.arange(1, len(names[i].text.split())):
            name = name + " " + names[i].text.split()[j]
        name_list.append(name)
    return name_list

In [82]:
data = pd.DataFrame()
for num_pag in range(6):
    link = "https://scrapethissite.com/pages/forms/?page_num="+str(num_pag+1)+"&per_page=100"

    source= requests.get(link).text

    soup=BeautifulSoup(source,'html.parser')

    table = soup.find_all('table',class_='table')[0]

    names = table.find_all('td', class_='name')
    years = table.find_all('td', class_='year')
    wins = table.find_all('td', class_='wins')
    losses = table.find_all('td', class_='losses')
    ot_losses = table.find_all('td', class_='ot-losses')
    win_rates = table.find_all('td', class_ ='pct text-success')
    gfs = table.find_all('td', class_= 'gf')
    gas = table.find_all('td', class_= 'ga')
    diffs = table.find_all('td', class_= 'diff text-success')


    names_list = []
    years_list = []
    wins_list = []
    losses_list = []
    ot_list = []
    gfs_list = []
    gas_list = []

    names_list = clean_names(names)   

    for i in range(len(names)):
        years_list.append(int(years[i].text))
        wins_list.append(int(wins[i].text))
        losses_list.append(int(losses[i].text))
        gfs_list.append(int(gfs[i].text))
        gas_list.append(int(gas[i].text))

        try:
            ot_list.append(ot_losses[i].text.replace("\n","").replace(" ", ""))
        except:
            ot_list.append('NaN')

    data_keys = {"Team": names_list, 'Year': years_list, 'OT Losses': ot_list, 
                "GF": gfs_list, "GA": gas_list, "Wins": wins_list, "Losses": losses_list}

    data = pd.concat([data, pd.DataFrame.from_dict(data_keys)], ignore_index=True)


In [83]:
data["Win_rate"] = (data["Wins"] / (data["Wins"] + data["Losses"]))
data["Diff"] = data["GF"] - data["GA"]


In [ ]:
# Uncomment line below to check data from web scrapping:
# data

### ESPN data

In [68]:
# Reading ESPN data
data_espn = pd.read_table("DataSet ESPN.csv", sep=",", index_col=[0])

In [ ]:
# Uncomment line below to check ESPN data:
# data_espn

# Data Exploration

Let me be clear:

1. I know nothing about ice hockey and NHL (actually, few things that I know come perhaps from watching Mighty Ducks as a kid...)
2. I know almost nothing about these two datasets

Therefore, one reading this notebook will join me in this journey of discovering ice hockey through data.


Ok, let's recap what we have:

1. data: dataframe containing information from https://www.scrapethissite.com/pages/forms 
2. data_espn: dataframe containing information from http://www.espn.com/nhl/statistics

And this is the information that I have about the ESPN data columns:

1. Team: name of the team
2. Home Games: number of games at home 
3. Home Total: total number of fans (all games in that year) at home games
4. Home Average: mean number of fans per home game 
5. Road Games: number of games played out of home 
6. Road Average: mean number of fans per game played out of home
7. Overall Games: total number of games played
8. Overall Average: mean number of fans per game (considering all games played)
9. Year: year that the season has ended 
10. Save Percentage: percentage of saves (i.e. avoiding score from the opposing team)
11. Penalty Minutes: minutes of penalty that the team has suffered
12. Penalty Minutes Against: minutes of penalty that the opposing team has suffered



## Cross-matching the datasets

First of all, I want to check if the two datasets can be cross-matched, i.e. if the two datasets comprises information from the same team in the same year season.

In [76]:
data.Year.value_counts()

2000    30
2001    30
2010    30
2009    30
2008    30
2007    30
2006    30
2005    30
2003    30
2002    30
2011    30
1999    28
1998    27
1997    26
1996    26
1995    26
1994    26
1993    26
1992    24
1991    22
1990    21
Name: Year, dtype: int64

In [71]:
data_espn.Year.value_counts()

2001    29
2002    29
2003    29
2004    29
2006    29
2007    29
2008    29
2009    29
2010    29
2011    29
Name: Year, dtype: int64

In [77]:
data_espn.

Buffalo Sabres             10
Toronto Maple Leafs        10
Philadelphia Flyers        10
Ottawa Senators            10
St Louis Blues             10
Vancouver Canucks          10
Washington Capitals        10
Pittsburgh Penguins        10
Montreal Canadiens         10
Calgary Flames             10
San Jose Sharks            10
Colorado Avalanche         10
Florida Panthers           10
Los Angeles Kings          10
Edmonton Oilers            10
Tampa Bay Lightning        10
Carolina Hurricanes        10
New Jersey Devils          10
Mighty Ducks of Anaheim    10
New York Islanders         10
Minnesota North Stars      10
Chicago Blackhawks         10
Dallas Stars               10
Columbus Blue Jackets      10
New York Rangers           10
Detroit Red Wings          10
Boston Bruins              10
Nashville Predators        10
Phoenix Coyotes            10
Name: Team, dtype: int64

In [86]:
data[data["Team"]== "Montreal Canadiens"]

,Team,Year,OT Losses,GF,GA,Wins,Losses,Win_rate,Diff
9,Montreal Canadiens,1990,,273,249,39,30,0.565217,24
30,Montreal Canadiens,1991,,267,207,41,28,0.594203,60
52,Montreal Canadiens,1992,,326,280,48,30,0.615385,46
78,Montreal Canadiens,1993,,283,248,41,29,0.585714,35
104,Montreal Canadiens,1994,,125,148,18,23,0.439024,-23
131,Montreal Canadiens,1995,,265,248,40,32,0.555556,17
157,Montreal Canadiens,1996,,249,276,31,36,0.462687,-27
183,Montreal Canadiens,1997,,235,208,37,32,0.536232,27
209,Montreal Canadiens,1998,,184,209,32,39,0.450704,-25
237,Montreal Canadiens,1999,4,196,194,35,34,0.507246,2


In [89]:
data_espn[data_espn["Team"]== "Montreal Canadiens"][["Team", "Year"]]

,Team,Year
0,Montreal Canadiens,2001
30,Montreal Canadiens,2002
58,Montreal Canadiens,2003
87,Montreal Canadiens,2004
116,Montreal Canadiens,2006
145,Montreal Canadiens,2007
174,Montreal Canadiens,2008
204,Montreal Canadiens,2009
233,Montreal Canadiens,2010
262,Montreal Canadiens,2011
